In [7]:
#@title Imports

!pip install igraph

import itertools
import random
import numpy as np
import random
import warnings
import numpy as np
import igraph as ig
import networkx as nx
from collections import Counter

In [4]:
#@title Generators interface

def is_latin_rectangle(rows):
    valid = True
    for row in rows:
        if len(set(row)) < len(row):
            valid = False
    if valid and rows:
        for i, val in enumerate(rows[0]):
            col = [row[i] for row in rows]
            if len(set(col)) < len(col):
                valid = False
                break
    return valid

def is_latin_square(rows):
    return is_latin_rectangle(rows) and len(rows) == len(rows[0])

def latin_square(items, shuffle=True):
    result = []
    for elems in itertools.permutations(items):
        valid = True
        for i, elem in enumerate(elems):
            orthogonals = [x[i] for x in result] + [elem]
            if len(set(orthogonals)) < len(orthogonals):
                valid = False
                break
        if valid:
            result.append(elems)
    if shuffle:
        random.shuffle(result)
    return result

def gen_srg(n):
 items = list(range(1, n + 1))
 random.shuffle(items)
 rows = latin_square(items)

 if not is_latin_square(rows):
   print("Invalid latin square generated")
   return None

 G = nx.Graph()
 for x, y in itertools.combinations(list(itertools.product(range(0, n), range(0, n))), 2):
  if x[0] == y[0] or x[1] == y[1] or rows[x[0]][x[1]] == rows[y[0]][y[1]]:
    G.add_edge((x[0],x[1]),(y[0],y[1]))

 if not nx.is_strongly_regular(G):
  print("Not strongly regular graph generated")
  return None

 return G

def generate_strongly_regular_problem(s = 4, isomorphic = False):
  G = ig.Graph.from_networkx(gen_srg(s))
  result = permutate(G, isomorphic)
  if result is None:
    while True:
      H = ig.Graph.from_networkx(gen_srg(s))
      if not G.isomorphic(H):
        return G, permutate(H, True)
  return G, result

def generate_regular_problem(n = 4, m = 16, isomorphic = False):
  G = ig.Graph.from_networkx(nx.random_regular_graph(n, m))
  result = permutate(G, isomorphic)
  if result is None:
    while True:
      H = ig.Graph.from_networkx(nx.random_regular_graph(n, m))
      if not G.isomorphic(H):
        return G, permutate(H, True)
  return G, result

def permutate(G, isomorphic = False):
  mapping = G.vs.indices
  random.shuffle(mapping)
  G.permute_vertices(mapping)
  A = G.get_edgelist()
  for i, e in enumerate(A):
    if random.random() < 0.5:
     A[i] = (e[1], e[0])

  random.shuffle(A)
  G = nx.Graph(A)

  if isomorphic:
    return ig.Graph.from_networkx(G)
  else:
    H = G.copy()
    G = ig.Graph.from_networkx(G)
    nx.double_edge_swap(H, 1, max_tries = 1000)
    H = ig.Graph.from_networkx(H)
    if not G.isomorphic(H):
      return H
    return None

def SuR():
  S = nx.Graph()
  S.add_edges_from([(0, 3), (0, 12), (0, 1), (0, 4), (1, 2), (1, 5), (1, 13), (2, 3), (2, 6), (2, 14), (3, 15), (3, 7)])
  S.add_edges_from([(4, 7), (4, 5), (4, 8), (5, 6), (5, 9), (6, 7), (6, 10), (7, 11)])
  S.add_edges_from([(8, 11), (8, 9), (8, 12), (9, 10), (9, 13), (10, 11), (10, 14), (11, 15)])
  S.add_edges_from([(12, 15), (12, 13), (13, 14), (14, 15)])
  return S

def rook():
  S = SuR()
  S.add_edges_from([(0, 2), (0, 8), (1, 3), (1, 9), (2, 10), (3, 11)])
  S.add_edges_from([(4, 6), (4, 12), (5, 7), (5, 13), (6, 14), (7, 15)])
  S.add_edges_from([(8, 10), (9, 11)])
  S.add_edges_from([(12, 14), (13, 15)])
  return permutate(ig.Graph.from_networkx(S), isomorphic = True)

def shrikhande():
  S = SuR()
  S.add_edges_from([(0, 15), (0, 5), (1, 12), (1, 6), (2, 13), (2, 7), (3, 4), (3, 14)])
  S.add_edges_from([(4, 9), (5, 10), (6, 11), (7, 8)])
  S.add_edges_from([(8, 13), (9, 14), (10, 15), (11, 12)])
  return permutate(ig.Graph.from_networkx(S), isomorphic = True)

def foolproof(G, H):
   if G.ecount() < 2 or H.ecount() < 2 or G.vcount() < 2 or H.vcount() < 2:
    return False
   H.vs.select(_degree=0).delete()
   G.vs.select(_degree=0).delete()
   if G.vcount() != H.vcount() or G.ecount() != H.ecount():
    return False
   if sorted(G.degree()) != sorted(H.degree()):
    return False
   return True

In [5]:
def diffactor(G):

    shire = {}

    for x,y,z in itertools.combinations(G.vs.indices, 3):

      teamX = set(G.neighbors(x)) - set(G.neighbors(y)) - set(G.neighbors(z))
      teamY = set(G.neighbors(y)) - set(G.neighbors(x)) - set(G.neighbors(z))
      teamZ = set(G.neighbors(z)) - set(G.neighbors(y)) - set(G.neighbors(x))
      teamM = set(G.neighbors(x)) & set(G.neighbors(y)) & set(G.neighbors(z))

      teamX.add(x)
      teamY.add(y)
      teamZ.add(z)

      prev = len(teamX) + len(teamY) + len(teamZ) + len(teamM)
      total = prev

      while prev < G.vcount():
       tx = teamX.copy()
       ty = teamY.copy()
       tz = teamZ.copy()

       for peace in teamM.copy():
        n = set(G.neighbors(peace))
        a = len(n & tx)
        b = len(n & ty)
        c = len(n & tz)

        tr = [a, b, c]
        if len(set(tr)) == 3:
         if tr.index(max(a, b, c)) == 0:
          teamX.add(peace)
          teamM.remove(peace)
         elif tr.index(max(a, b, c)) == 1:
          teamY.add(peace)
          teamM.remove(peace)
         elif tr.index(max(a, b, c)) == 2:
          teamZ.add(peace)
          teamM.remove(peace)

       tx = teamX.copy()
       ty = teamY.copy()
       tz = teamZ.copy()

       for free in set(G.vs.indices) - (tx | ty | tz | teamM):
        n = set(G.neighbors(free))
        a = len(n & tx)
        b = len(n & ty)
        c = len(n & tz)

        tr = [a, b, c]
        if len(set(tr)) == 3:
         if tr.index(max(a, b, c)) == 0:
          teamX.add(free)
         elif tr.index(max(a, b, c)) == 1:
          teamY.add(free)
         elif tr.index(max(a, b, c)) == 2:
          teamZ.add(free)

       total = len(teamX) + len(teamY) + len(teamM) + len(teamZ)
       if total == prev:
        break
       else:
        prev = total
        break

      shire[(x,y,z)] = (tuple(sorted([len(teamY), len(teamX), len(teamZ)])), total)

    return frozenset(Counter(shire.values()).most_common())

In [8]:
def iso(G, H, iso = False):
  if foolproof(G, H):
    assert((diffactor(G) == diffactor(H)) == iso)

In [9]:
isomorphic = False

G = shrikhande()
H = rook()

for _ in range(0, 1):
  G = shrikhande()
  H = rook()
  iso(G, H, iso = isomorphic)

In [21]:
isomorphic = False

for _ in range(0, 10):
  G, H = generate_strongly_regular_problem(4, isomorphic = isomorphic)
  iso(G, H, iso = isomorphic)

In [19]:
isomorphic = False

for _ in range(0, 1000):
  G, H = generate_regular_problem(3, 12, isomorphic = isomorphic)
  iso(G, H, iso = isomorphic)